In [1]:
import os
import numpy as np
import pandas as pd
from math import sqrt

# ======================== 配置区域 ========================

# 当前 Notebook 所在目录，默认就是遍历当前目录
BASE_DIR = "./results/solar/"

# 输出的汇总 CSV 文件名
OUTPUT_CSV = "eval_summary_all_experiments.csv"

# 各数据集/站点的容量 Cap（单位与 y_pred/y_true 一致）
# 注意：GEFCOM 已经归一化，Cap = 1
CAP_SITE = {
    "SKIPPD": 30,
    "CSGS1": 50,
    "CSGS2": 130,
    "CSGS3": 30,
    "CSGS4": 130,
    "CSGS5": 110,
    "CSGS6": 35,
    "CSGS7": 30,
    "CSGS8": 30,
}

CAP_GEFCOM = 1.0  # GEFCOM 全部通道统一用 Cap=1

# 各数据集家族对应的时间分辨率（分钟）
# 家族判断规则见 infer_family_and_resolution 函数
DATASET_RESOLUTION = {
    "GEFCOM": 60,   # 1h
    "CSG": 15,      # 15min （CSGS* 归入 CSG 家族）
    "SKIPPD": 15,   # 15min
}

# track 类型分类时的容差（小时），防止浮点误差
HOUR_EPS = 1e-6

print("配置已加载。可以根据需要修改 CAP_SITE / CAP_GEFCOM / DATASET_RESOLUTION 等。")

配置已加载。可以根据需要修改 CAP_SITE / CAP_GEFCOM / DATASET_RESOLUTION 等。


In [2]:
import os

prefix_old = "GEFCom2014_Task"
prefix_new = "GEFCOM_TASK"

base_path = os.path.abspath(BASE_DIR)
print("检查目录：", base_path)

renamed = []

for name in os.listdir(base_path):
    old_path = os.path.join(base_path, name)
    if not os.path.isdir(old_path):
        continue

    if name.startswith(prefix_old):
        new_name = prefix_new + name[len(prefix_old):]
        new_path = os.path.join(base_path, new_name)

        if os.path.exists(new_path):
            print(f"[跳过] 目标目录已存在，不重命名：{name} -> {new_name}")
            continue

        os.rename(old_path, new_path)
        renamed.append((name, new_name))
        print(f"[重命名] {name} -> {new_name}")

if not renamed:
    print("没有找到以 'GEFCom2014_Task' 开头的目录。")
else:
    print("已完成重命名，共处理目录数：", len(renamed))

检查目录： /home/liym/solar_energy/results/solar
没有找到以 'GEFCom2014_Task' 开头的目录。


In [3]:
def parse_experiment_name(folder_name):
    """
    从文件夹名中解析出：
      - dataset_site_base: 例如 CSGS2, GEFCOM_TASK1, SKIPPD
      - seq_len: int
      - pred_len: int
      - model: 例如 PatchTST, iTransformer
    规则：找到第一个可以转为 int 的字段作为 seq_len。
    """
    parts = folder_name.split("_")
    first_int_idx = None
    for i, p in enumerate(parts):
        try:
            _ = int(p)
            first_int_idx = i
            break
        except ValueError:
            continue

    if first_int_idx is None or first_int_idx + 2 >= len(parts):
        raise ValueError(f"无法从文件夹名中解析 seq_len/pred_len/model: {folder_name}")

    dataset_site_base = "_".join(parts[:first_int_idx])
    seq_len = int(parts[first_int_idx])
    pred_len = int(parts[first_int_idx + 1])
    model = parts[first_int_idx + 2]

    return dataset_site_base, seq_len, pred_len, model


def infer_family_and_resolution(dataset_site_base):
    """
    根据 dataset_site_base 推断数据集家族和时间分辨率（分钟）。
    规则：
      - 以 'GEFCOM' 开头 -> family='GEFCOM', resolution=60
      - 以 'CSG' 开头（例如 CSGS1..8）-> family='CSG', resolution=15
      - 以 'SKIPPD' 开头 -> family='SKIPPD', resolution=15
    """
    if dataset_site_base.startswith("GEFCOM"):
        family = "GEFCOM"
    elif dataset_site_base.startswith("CSG"):
        family = "CSG"
    elif dataset_site_base.startswith("SKIPPD"):
        family = "SKIPPD"
    else:
        raise ValueError(f"无法根据站点名判断数据集家族: {dataset_site_base}")

    if family not in DATASET_RESOLUTION:
        raise ValueError(f"DATASET_RESOLUTION 中缺少家族 {family} 的配置")

    resolution_min = DATASET_RESOLUTION[family]
    return family, resolution_min


def classify_track_type(pred_len, resolution_min):
    """
    根据 pred_len 和时间分辨率（分钟）推断当前实验属于哪种 track：
      - 'one-step'  : 仅一步预测（15min 或 1h）
      - '4h'        : 预测到 4 小时
      - '72h'       : 预测到 72 小时（短期 track），评估时只看 D+1 点（24h）
    """
    horizon_hours = pred_len * resolution_min / 60.0

    if abs(horizon_hours - 72.0) <= HOUR_EPS:
        return "72h", horizon_hours

    if abs(horizon_hours - 4.0) <= HOUR_EPS:
        return "4h", horizon_hours

    if horizon_hours <= 2.0 + HOUR_EPS:
        return "one-step", horizon_hours

    raise ValueError(
        f"无法分类 track_type，pred_len={pred_len}, resolution_min={resolution_min}, horizon_hours={horizon_hours:.4f} 不在 one-step/4h/72h 范围内"
    )


def get_cap_for_experiment(dataset_site_base, family):
    """
    根据站点名和家族返回 Cap。
    - GEFCOM 家族统一用 CAP_GEFCOM = 1
    - 其它家族使用 CAP_SITE[dataset_site_base]
    """
    if family == "GEFCOM":
        return CAP_GEFCOM
    
    # 1) 先直接用完整名字找
    if dataset_site_base in CAP_SITE:
        return CAP_SITE[dataset_site_base]

    # 2) 如果有下划线后缀（例如 CSGS3_S / CSGS3_MS），尝试用下划线前的部分匹配
    if "_" in dataset_site_base:
        core = dataset_site_base.split("_")[0]
        if core in CAP_SITE:
            return CAP_SITE[core]

    # 3) 仍然找不到就报错
    raise KeyError(f"未在 CAP_SITE 中找到站点 {dataset_site_base} 对应的 Cap 配置")


def compute_mae_rmse(pred_slice, true_slice):
    """
    计算给定一维预测/真实数组的 MAE 和 RMSE，自动去掉 NaN。
    """
    pred_slice = np.asarray(pred_slice).reshape(-1)
    true_slice = np.asarray(true_slice).reshape(-1)
    mask = (~np.isnan(pred_slice)) & (~np.isnan(true_slice))
    if mask.sum() == 0:
        return np.nan, np.nan, 0
    diff = pred_slice[mask] - true_slice[mask]
    mae = np.mean(np.abs(diff))
    rmse = sqrt(np.mean(diff ** 2))
    return mae, rmse, int(mask.sum())

print("解析与分类工具函数已定义。")

解析与分类工具函数已定义。


In [4]:
rows = []

print("开始遍历目录:", os.path.abspath(BASE_DIR))
subdirs = sorted([d for d in os.listdir(BASE_DIR) if os.path.isdir(os.path.join(BASE_DIR, d))])

print("发现子目录数量:", len(subdirs))

for d in subdirs:
    exp_dir = os.path.join(BASE_DIR, d)
    y_pred_path = os.path.join(exp_dir, "y_pred.npy")
    y_true_path = os.path.join(exp_dir, "y_true.npy")

    if not (os.path.exists(y_pred_path) and os.path.exists(y_true_path)):
        continue

    try:
        dataset_site_base, seq_len, pred_len, model = parse_experiment_name(d)
    except Exception as e:
        print(f"[警告] 解析文件夹名失败 {d}: {e}")
        continue

    try:
        family, resolution_min = infer_family_and_resolution(dataset_site_base)
    except Exception as e:
        print(f"[警告] 判断家族/时间分辨率失败 {d}: {e}")
        continue

    try:
        track_type, horizon_hours = classify_track_type(pred_len, resolution_min)
    except Exception as e:
        print(f"[警告] track 类型分类失败 {d}: {e}")
        continue

    try:
        cap = get_cap_for_experiment(dataset_site_base, family)
    except KeyError as e:
        print(f"[警告] 未找到 Cap 配置 {d}: {e}")
        cap = np.nan

    try:
        y_pred = np.load(y_pred_path)
        y_true = np.load(y_true_path)
    except Exception as e:
        print(f"[警告] 加载 {d} 的 y_pred/y_true 失败: {e}")
        continue

    if y_pred.shape != y_true.shape:
        print(f"[警告] {d}: y_pred.shape={y_pred.shape} 与 y_true.shape={y_true.shape} 不一致，跳过")
        continue

    if y_pred.ndim == 2:
        y_pred = y_pred[:, :, np.newaxis]
        y_true = y_true[:, :, np.newaxis]
    elif y_pred.ndim != 3:
        print(f"[警告] {d}: y_pred 维度 {y_pred.ndim} 不在 [2,3]，跳过")
        continue

    N, pl, C = y_pred.shape
    if pl != pred_len:
        print(f"[警告] {d}: 文件名中的 pred_len={pred_len} 与 y_pred.shape[1]={pl} 不一致，使用实际的 {pl}")
        pred_len = pl
        try:
            track_type, horizon_hours = classify_track_type(pred_len, resolution_min)
        except Exception as e:
            print(f"[警告] {d}: 根据实际 pred_len 重新分类 track_type 失败: {e}")
            continue

    for c in range(C):
        y_pred_c = y_pred[:, :, c]
        y_true_c = y_true[:, :, c]

        if family == "GEFCOM":
            channel_idx = c + 1
            site_for_csv = f"{dataset_site_base}_ch{channel_idx}"
        else:
            channel_idx = c + 1
            site_for_csv = dataset_site_base

        eval_desc = ""
        if track_type == "one-step":
            pred_slice = y_pred_c[:, -1]
            true_slice = y_true_c[:, -1]
            if resolution_min == 15:
                eval_desc = "1-step (15min)"
            elif resolution_min == 60:
                eval_desc = "1-step (1h)"
            else:
                eval_desc = f"1-step ({resolution_min}min)"

        elif track_type == "4h":
            pred_slice = y_pred_c[:, -1]
            true_slice = y_true_c[:, -1]
            eval_desc = "4h ahead (last step)"

        elif track_type == "72h":
            pred_slice = y_pred_c[:, -1]
            true_slice = y_true_c[:, -1]
            eval_desc = "D+1 (24h ahead)"

        else:
            print(f"[警告] {d}, 通道{channel_idx}: 未知 track_type={track_type}，跳过")
            continue

        mae, rmse, num_samples = compute_mae_rmse(pred_slice, true_slice)
        if num_samples == 0:
            print(f"[警告] {d}, 通道{channel_idx}: 有效样本数为 0，跳过")
            continue

        if np.isnan(cap):
            acc_mae = np.nan
            acc_rmse = np.nan
        else:
            acc_mae = 1.0 - mae / cap
            acc_rmse = 1.0 - rmse / cap

        row = {
            "exp_name": d,
            "dataset_site_base": dataset_site_base,
            "family": family,
            "resolution_min": resolution_min,
            "model": model,
            "seq_len": seq_len,
            "pred_len": pred_len,
            "track_type": track_type,
            "eval_horizon_desc": eval_desc,
            "channel": channel_idx,
            "site_for_csv": site_for_csv,
            "cap": cap,
            "num_samples": num_samples,
            "mae": mae,
            "rmse": rmse,
            "acc_mae": acc_mae,
            "acc_rmse": acc_rmse,
        }
        rows.append(row)

print("遍历完成。共收集到行数:", len(rows))

if rows:
    df_out = pd.DataFrame(rows)
    df_out.to_csv(OUTPUT_CSV, index=False)
    print(f"结果已保存到: {os.path.abspath(OUTPUT_CSV)}")
    display(df_out.head())
else:
    print("未生成任何评估结果，请检查目录结构和配置。")

开始遍历目录: /home/liym/solar_energy/results/solar
发现子目录数量: 7
[警告] GEFCOM_TASK15_336_4_GBDT_custom_solar_ftM_sl336_ll48_pl4_dm512_nh8_el2_dl1_df2048_fc1_ebtimeF_dtTrue_Exp_0: 文件名中的 pred_len=4 与 y_pred.shape[1]=1 不一致，使用实际的 1
[警告] GEFCOM_TASK15_336_72_GBDT_custom_solar_ftM_sl336_ll48_pl72_dm512_nh8_el2_dl1_df2048_fc1_ebtimeF_dtTrue_Exp_0: 文件名中的 pred_len=72 与 y_pred.shape[1]=1 不一致，使用实际的 1
[警告] 解析文件夹名失败 lgb_test_single: 无法从文件夹名中解析 seq_len/pred_len/model: lgb_test_single
遍历完成。共收集到行数: 18
结果已保存到: /home/liym/solar_energy/eval_summary_all_experiments.csv


,exp_name,dataset_site_base,family,resolution_min,model,seq_len,pred_len,track_type,eval_horizon_desc,channel,site_for_csv,cap,num_samples,mae,rmse,acc_mae,acc_rmse
0,GEFCOM_TASK15_336_1_DLinear_custom_solar_ftM_s...,GEFCOM_TASK15,GEFCOM,60,DLinear,336,1,one-step,1-step (1h),1,GEFCOM_TASK15_ch1,1.0,5696,0.109796,0.148525,0.890204,0.851475
1,GEFCOM_TASK15_336_1_DLinear_custom_solar_ftM_s...,GEFCOM_TASK15,GEFCOM,60,DLinear,336,1,one-step,1-step (1h),2,GEFCOM_TASK15_ch2,1.0,5696,0.110065,0.147939,0.889935,0.852061
2,GEFCOM_TASK15_336_1_DLinear_custom_solar_ftM_s...,GEFCOM_TASK15,GEFCOM,60,DLinear,336,1,one-step,1-step (1h),3,GEFCOM_TASK15_ch3,1.0,5696,0.104309,0.139665,0.895692,0.860335
3,GEFCOM_TASK15_336_1_GBDT_custom_solar_ftM_sl33...,GEFCOM_TASK15,GEFCOM,60,GBDT,336,1,one-step,1-step (1h),1,GEFCOM_TASK15_ch1,1.0,5696,0.217601,0.269721,0.782399,0.730279
4,GEFCOM_TASK15_336_1_GBDT_custom_solar_ftM_sl33...,GEFCOM_TASK15,GEFCOM,60,GBDT,336,1,one-step,1-step (1h),2,GEFCOM_TASK15_ch2,1.0,5696,0.237986,0.291393,0.762014,0.708607
